# 텐서플로우를 이용한 ANN(Artificial Neural Network) 구현

In [1]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

MNIST 데이터를 다운로드합니다.

In [2]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

이미지들을 float32 데이터 타입으로 변경합니다.

In [3]:
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

28x28 형태의 이미지를 784 차원으로 flattenting합니다.

In [4]:
x_train, x_test = x_train/255., x_test/255.

[0,255] 사이의 값을 [0,1] 사이의 값으로 Normalize합니다

In [5]:
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

레이블 데이터에 one-hot encoding을 적용합니다.

In [6]:
learning_rate = 0.001
num_epochs = 30 #학습횟수
batch_size = 256 #배치개수
display_step = 1 #손실함수 출력 주기
input_size = 784 # 28 * 28
hidden1_size = 256
hidden2_size = 256
output_size = 10

학습을 위한 설정값들을 정의합니다.

In [7]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.

In [8]:
# ANN 모델을 정의한다.
class ANN(object):
    # ANN 모델을 위한 tf.Variable들을 정의한다.
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal(shape=[input_size, hidden1_size]))
        self.b1 = tf.Variable(tf.random.normal(shape=[hidden1_size]))
        self.W2 = tf.Variable(tf.random.normal(shape=[hidden1_size, hidden2_size]))
        self.b2 = tf.Variable(tf.random.normal(shape=[hidden2_size]))
        self.W_output = tf.Variable(tf.random.normal(shape=[hidden2_size, output_size]))
        self.b_output = tf.Variable(tf.random.normal(shape=[output_size]))
    def __call__(self, x):
        H1_output = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        H2_output = tf.nn.relu(tf.matmul(H1_output, self.W2) + self.b2)
        logits = tf.matmul(H2_output, self.W_output) + self.b_output
        
        return logits

In [9]:
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = y))

cross-entropy 손실 함수를 정의한다.

In [10]:
optimizer = tf.optimizers.Adam(learning_rate)

최적화를 위한 Adam 옵티마이저를 정의한다.

In [11]:
@tf.function
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = cross_entropy_loss(y_pred, y)
    gradients = tape.gradient(loss, vars(model).values())
    optimizer.apply_gradients(zip(gradients, vars(model).values()))

최적화를 위한 function을 정의한다.

In [12]:
@tf.function
def compute_accuracy(y_pred, y):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

모델의 정확도를 출력하는 함수를 정의한다.

In [13]:
ANN_model = ANN()

ANN 모델을 선언한다.

In [14]:
# 지정된 횟수만큼 최적화를 수행한다.
for epoch in range(num_epochs):
    average_loss = 0.
    total_batch = int(x_train.shape[0] / batch_size)
    # 모든 배치들에 대해서 최적화를 수행한다.
    for batch_x, batch_y in train_data:
        # 옵티마이저를 실행해서 파라미터들을 업데이트한다.
        _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
        # 평균 손실을 측정한다.
        average_loss += current_loss / total_batch
    # 지정된 epoch마다 학습결과를 출력한다.
    if epoch %display_step ==0:
        print("반복(Epoch): %d, 손실 함수(Loss): %f"%((epoch+1), average_loss))
        

반복(Epoch): 1, 손실 함수(Loss): 213.309570
반복(Epoch): 2, 손실 함수(Loss): 55.734787
반복(Epoch): 3, 손실 함수(Loss): 36.032207
반복(Epoch): 4, 손실 함수(Loss): 26.685520
반복(Epoch): 5, 손실 함수(Loss): 20.737633
반복(Epoch): 6, 손실 함수(Loss): 16.734838
반복(Epoch): 7, 손실 함수(Loss): 13.517226
반복(Epoch): 8, 손실 함수(Loss): 11.106730
반복(Epoch): 9, 손실 함수(Loss): 9.082035
반복(Epoch): 10, 손실 함수(Loss): 7.573379
반복(Epoch): 11, 손실 함수(Loss): 6.245502
반복(Epoch): 12, 손실 함수(Loss): 5.077939
반복(Epoch): 13, 손실 함수(Loss): 4.199845
반복(Epoch): 14, 손실 함수(Loss): 3.486521
반복(Epoch): 15, 손실 함수(Loss): 2.861541
반복(Epoch): 16, 손실 함수(Loss): 2.299628
반복(Epoch): 17, 손실 함수(Loss): 1.865000
반복(Epoch): 18, 손실 함수(Loss): 1.466027
반복(Epoch): 19, 손실 함수(Loss): 1.232642
반복(Epoch): 20, 손실 함수(Loss): 0.978890
반복(Epoch): 21, 손실 함수(Loss): 0.786336
반복(Epoch): 22, 손실 함수(Loss): 0.639179
반복(Epoch): 23, 손실 함수(Loss): 0.514994
반복(Epoch): 24, 손실 함수(Loss): 0.424804
반복(Epoch): 25, 손실 함수(Loss): 0.330436
반복(Epoch): 26, 손실 함수(Loss): 0.252245
반복(Epoch): 27, 손실 함수(Loss): 0.214588
반

In [15]:
print("정확도(Accuracy): %f"% compute_accuracy(ANN_model(x_test), y_test))

정확도(Accuracy): 0.947000


테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력한다.  
정확도: 약 94%